# ch 6. naive bayes를 이용한 텍스트 분류

이번 챕터에서는 scikit learn에 내장된 NaiveBayesClassifier를 이용해서 실제 텍스트를 분류해보고, 성능을 측정해보겠습니다. 

## 데이터 셋 준비 

In [1]:
import pandas as pd

train_df = pd.read_csv("./data/naver_reviews_train.csv")
test_df = pd.read_csv("./data/naver_reviews_test.csv")

In [2]:
train_df

,document,label
0,아 더빙.. 진짜 짜증나네요 목소리,0
1,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,너무재밓었다그래서보는것을추천한다,0
3,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
...,...,...
149995,인간이 문제지.. 소는 뭔죄인가..,0
149996,평점이 너무 낮아서...,1
149997,이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?,0
149998,청춘 영화의 최고봉.방황과 우울했던 날들의 자화상,1


In [3]:
test_df

,document,label
0,굳 ㅋ,1
1,GDNTOPCLASSINTHECLUB,0
2,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0
...,...,...
49995,오랜만에 평점 로긴했네ㅋㅋ 킹왕짱 쌈뽕한 영화를 만났습니다 강렬하게 육쾌함,1
49996,의지 박약들이나 하는거다 탈영은 일단 주인공 김대희 닮았고 이등병 찐따 OOOO,0
49997,그림도 좋고 완성도도 높았지만... 보는 내내 불안하게 만든다,0
49998,절대 봐서는 안 될 영화.. 재미도 없고 기분만 잡치고.. 한 세트장에서 다 해먹네,0


### 결측치 확인 및 제거

In [9]:
train_df.isnull().sum()

document    0
label       0
dtype: int64

In [10]:
test_df.isnull().sum()

document    0
label       0
dtype: int64

In [7]:
train_df = train_df.dropna()

In [8]:
test_df = test_df.dropna()

### 토큰화

In [11]:
from konlpy.tag import Komoran

komoran = Komoran()

In [12]:
from tqdm import tqdm

tqdm.pandas()
train_df["tokens"] = train_df["document"].progress_apply(lambda x: komoran.morphs(x))
test_df["tokens"] = test_df["document"].progress_apply(lambda x: komoran.morphs(x))

100%|█████████████████████████████████████████████████████████████████████| 149995/149995 [00:51<00:00, 2939.05it/s]
/var/folders/wx/2vw1xwz972zdxff5h8gzrml00000gn/T/ipykernel_52913/145535669.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df["tokens"] = train_df["document"].progress_apply(lambda x: komoran.morphs(x))
100%|███████████████████████████████████████████████████████████████████████| 49997/49997 [00:17<00:00, 2896.69it/s]
/var/folders/wx/2vw1xwz972zdxff5h8gzrml00000gn/T/ipykernel_52913/145535669.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/sta

In [14]:
test_df

,document,label,tokens
0,굳 ㅋ,1,"[굳, ㅋ]"
1,GDNTOPCLASSINTHECLUB,0,[GDNTOPCLASSINTHECLUB]
2,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0,"[뭐, 야, 이, 평점, 들, 은, ..., ., 나쁘, 지, ㄴ, 않, 지만, 1..."
3,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0,"[지루, 하, 지, 는, 않, 은데, 완전, 막, 장임, ..., 돈, 주, 고, ..."
4,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0,"[3D, 만, 아니, 었, 어도, 별, 다섯, 개, 주, 었, 을, 텐, 데, .,..."
...,...,...,...
49995,오랜만에 평점 로긴했네ㅋㅋ 킹왕짱 쌈뽕한 영화를 만났습니다 강렬하게 육쾌함,1,"[오랜만, 에, 평점, 로긴했넼ㅋ, 킹왕짱, 쌈, 뽕, 하, ㄴ, 영화, 를, 만나..."
49996,의지 박약들이나 하는거다 탈영은 일단 주인공 김대희 닮았고 이등병 찐따 OOOO,0,"[의지, 박, 약, 들, 이나, 하, 는, 거, 다, 탈영, 은, 일단, 주인공, ..."
49997,그림도 좋고 완성도도 높았지만... 보는 내내 불안하게 만든다,0,"[그림, 도, 좋, 고, 완성도, 도, 높, 았, 지만, ..., 보, 는, 내내,..."
49998,절대 봐서는 안 될 영화.. 재미도 없고 기분만 잡치고.. 한 세트장에서 다 해먹네,0,"[절대, 보, 아서, 는, 안, 되, ㄹ, 영화, ., ., 재미, 도, 없, 고,..."


### CountVectorizer를 이용한 벡터화
Naive Bayes를 학습시키기 위해서는 문서 내 특정 단어의 빈도 수를 세어주어야 합니다. 이를 CountVectorizer를 이용해서 구현해보겠습니다.

In [15]:
from sklearn.feature_extraction.text import CountVectorizer

def dummy_fun(doc):
    return doc

vectorizer = CountVectorizer(
    analyzer="word",
    tokenizer=dummy_fun,
    preprocessor=dummy_fun,
    token_pattern=None
)

In [16]:
vectorizer.fit(train_df["tokens"])
x_train = vectorizer.transform(train_df["tokens"])
y_train = train_df["label"]
x_test = vectorizer.transform(test_df["tokens"])
y_test = test_df["label"]

## Naive Bayes Classifier 학습

scikit learn에 내장된 MultinomialNB를 이용하여 Naive Bayes Classifier를 학습시켜 보겠습니다. MultinomialNB를 사용하는 이유는 단어의 등장 빈도 수를 이용하여 분류하기 때문입니다. 먼저 학습시에 교차 검증으로 정확도를 측정하고, 전체 데이터 셋으로 다시 학습시킨 뒤, 테스트 셋에 대하여 정확도를 측정해보겠습니다.

In [20]:
from sklearn.model_selection import StratifiedKFold, cross_val_score

stratified_kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [21]:
from sklearn.naive_bayes import MultinomialNB 

model = MultinomialNB()
cross_val_scores = cross_val_score(model, x_train, y_train, cv=stratified_kfold)
print("교차 검증 정확도:", cross_val_scores)

교차 검증 정확도: [0.82666089 0.82946098 0.83222774 0.83076103 0.83149438]


In [22]:
model = MultinomialNB()
model.fit(x_train, y_train)

MultinomialNB()

In [24]:
from sklearn.metrics import accuracy_score

y_pred = model.predict(x_test)
accuracy = accuracy_score(y_test, y_pred)
print("테스트 셋 정확도:", accuracy)

테스트 셋 정확도: 0.82406944416665


## 정리

이번 챕터에서는 scikit learn에 내장된 MultinomialNB 클래스를 이용하여 텍스트 분류 모델을 학습시켜 보았습니다. 그 결과 82%로 상당히 준수한 성능이 나왔습니다. LSTM을 이용한 딥러닝 모델도 85% 정도 성능을 내는 것을 생각해보면 굉장히 가성비가 좋은 모델임을 알 수 있습니다. 베이즈 정리가 이렇게도 활용되는 구나 하는 점과 단순한 모델의 강력함을 꼭 기억하고 넘어갔으면 좋겠습니다.